In [1]:
import csv
import pandas as pd
import numpy as np
import math
import scipy
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [2]:
appVolDat = pd.read_csv('filingVolumes.csv')
prediction = appVolDat.iloc[1405]
appVolDat = appVolDat.iloc[1320:1404]
appVolDat = appVolDat.reset_index(drop=True)
maximum = max(appVolDat.volume)
appVolDat['vol'] = appVolDat.volume/maximum

In [3]:
for i in range(1,13):
    appVolDat['volShift'+str(i)] = appVolDat.vol.shift(i)
appVolDat = appVolDat.iloc[12:]

In [4]:
def func(X,a,b,c,d,e,f,g,h,i,j,k,l):
    return a*np.log(X['volShift1']) + b*np.log(X['volShift2']) + c*np.log(X['volShift3']) + \
d*np.log(X['volShift4']) + e*np.log(X['volShift5']) + f*np.log(X['volShift6']) + \
g*np.log(X['volShift7']) + h*np.log(X['volShift8']) + i*np.log(X['volShift9']) + \
j*np.log(X['volShift10']) + k*np.log(X['volShift11']) + l*np.log(X['volShift12'])

In [5]:
y = np.array(appVolDat['vol'])
fitParams, fitCovariances = curve_fit(func, appVolDat, y)
print(fitParams)

[ 0.11577073  0.0914659   0.2287173  -0.01392196  0.13095935  0.05109556
 -0.03557866 -0.2213096  -0.06797807 -0.37979694 -0.37671498 -0.22137395]


In [6]:
ypred = fitParams[0]*appVolDat['volShift1'] + fitParams[1]*appVolDat['volShift2'] + fitParams[2]*appVolDat['volShift3'] + \
fitParams[3]*appVolDat['volShift4'] + fitParams[4]*appVolDat['volShift5'] + fitParams[5]*appVolDat['volShift6'] \
+ fitParams[6]*appVolDat['volShift7'] + fitParams[7]*appVolDat['volShift8'] + fitParams[8]*appVolDat['volShift9'] \
+ fitParams[9]*appVolDat['volShift10'] + fitParams[10]*appVolDat['volShift11'] + fitParams[11]*appVolDat['volShift12']

In [7]:
appVolDat.insert(3, 'volPred', np.array([10**x for x in ypred]))

In [8]:
appVolDat['vol'] = appVolDat['vol'] * maximum
appVolDat['volPred'] = appVolDat['volPred'] * maximum
appVolDat

,month,volume,vol,volPred,volShift1,volShift2,volShift3,volShift4,volShift5,volShift6,volShift7,volShift8,volShift9,volShift10,volShift11,volShift12
12,1/1/2011,35259,35259.0,42418.868116,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785,0.518229,0.400055,0.377523
13,2/1/2011,35057,35057.0,38422.947351,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785,0.518229,0.400055
14,3/1/2011,45603,45603.0,40391.205316,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785,0.518229
15,4/1/2011,38163,38163.0,38816.451894,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785
16,5/1/2011,38934,38934.0,39805.836320,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541
17,6/1/2011,43581,43581.0,41529.235952,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566
18,7/1/2011,35761,35761.0,39354.294896,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340
19,8/1/2011,41170,41170.0,38844.411407,0.430435,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235
20,9/1/2011,47684,47684.0,41459.038637,0.495540,0.430435,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614
21,10/1/2011,37863,37863.0,38431.220298,0.573946,0.495540,0.430435,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334


In [9]:
np.sum(np.absolute(appVolDat['volPred']-appVolDat['vol']))

548208.1634356265

In [10]:
'''
All: 548208.1634356265
return a*np.log(X['volShift1']) + b*np.log(X['volShift2']) + c*np.log(X['volShift3']) + \
d*np.log(X['volShift4']) + e*np.log(X['volShift5']) + f*np.log(X['volShift6']) + \
g*np.log(X['volShift7']) + h*np.log(X['volShift8']) + i*np.log(X['volShift9']) + \
j*np.log(X['volShift10']) + k*np.log(X['volShift11']) + l*np.log(X['volShift12'])

1,2,12: 600460.476828005
return a*np.log(X['volShift1']) + b*np.log(X['volShift2']) + c*np.log(X['volShift12'])

'''

"\nAll: 548208.1634356265\nreturn a*np.log(X['volShift1']) + b*np.log(X['volShift2']) + c*np.log(X['volShift3']) + d*np.log(X['volShift4']) + e*np.log(X['volShift5']) + f*np.log(X['volShift6']) + g*np.log(X['volShift7']) + h*np.log(X['volShift8']) + i*np.log(X['volShift9']) + j*np.log(X['volShift10']) + k*np.log(X['volShift11']) + l*np.log(X['volShift12'])\n\n1,2,12: 600460.476828005\nreturn a*np.log(X['volShift1']) + b*np.log(X['volShift2']) + c*np.log(X['volShift12'])\n\n"